In [2]:
import sys
sys.path.append("../")
import npccv.fft as nft
import npccv.utils as utils
import numpy as np

class fftimg:
    def __init__(self,img):
        """
            img:tpye->numpy array,one channels
        """
        self.img = img
        self.fft2 = nft.FFT2(self.img,depart=False)

    def Power(self):
        """
        图像总功率
        """
        return np.sum(np.abs(self.fft2) ** 2)

    def __easyFDF__(self,d0 = 0.5):
        dist = utils.distMat(self.img)
        # huv = 

    FDFfuc = {
        "easy":""
    }

    #频域滤波
    def FDF(self,param,type = "easy"):
        """
        frequency domain filtering

        type:
        easy:理想低通滤波器
        """



        pass

    def FSD(self):
        """
            返回频谱,频域
        """
        res = np.fft.fftshift(self.fft2)

        stm = np.log(np.abs(res))
        pse = np.angle(res)

        return  stm,pse


